In [20]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, WebDriverException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from requests_html import HTMLSession
from csv import DictReader
from seleniumwire import webdriver
from user_agent import generate_user_agent, generate_navigator
from selenium_stealth import stealth

import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from requests.packages.urllib3 import disable_warnings

from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

from IPython.display import clear_output, display
from tqdm.auto import tqdm
import datetime
import traceback
import gc
import os
import time
import pandas as pd
import json
import dill
import re

In [21]:
def get_url_request(url):
    disable_warnings(InsecureRequestWarning)
    useragent_rotate = generate_user_agent()
    headers = {
        'User-Agent': useragent_rotate
    }
    
    response  = requests.get(url, headers=headers, verify=False)
    return response

In [2]:
def scrape_tempo_attraction_article(date_early=None, date_latest=None):
    def get_url_request(url):
        useragent_rotate = generate_user_agent()
        headers = {
            'User-Agent': useragent_rotate
        }
        
        response  = requests.get(url, headers=headers)
        return response

    url = 'https://www.tempo.co/indeks/{}-{}-{}/travel'
    article_information_dict = {}
    
    date_latest = datetime.date(2024, 1, 9)
    date_early = datetime.date(2014, 2, 3)
    dates = [date_latest - datetime.timedelta(days=x) for x in range((date_latest-date_early).days + 1)]

    url_logger = display(display_id=True)
    for date in tqdm(dates, desc='Looping Each Dates'):
        day, month, year = date.strftime('%d'), date.strftime('%m'), date.strftime('%Y')
        article_logger = display(display_id=True)
        url_logger.update(url.format(year, month, day))
    
        response = get_url_request(url.format(year, month, day))
        soup = bs(response.text)
        soup_article_links = soup.select('body > main > div.container.small.left.padding-no > main > div.card-box.ft240.margin-bottom-sm > article > h2 > a')
    
        if soup_article_links:
            article_information_dict.setdefault(year, {})\
                .setdefault(month, {})\
                .setdefault(day, {})
            for index, link in enumerate(tqdm(soup_article_links, desc=f'Looping Each Links of Dates: {year}-{month}-{day}', leave=False)):
                for i in range(15):
                    article_response = get_url_request(link['href'])
                    if article_response.status_code == 200:
                        break
                    elif article_response.status_code == 504:
                        time.sleep(0.4 * (i+1))
                article_soup = bs(article_response.text)
    
                article_title = article_soup.select_one('body > main > div.container.small.left.padding-no > main > article.detail-artikel > div.detail-title > h1').text
                article_description = ""
                for item in article_soup.select('#isi > div > div.detail-konten > p'):
                    article_description += item.text + ' '
    
                article_information_dict[year][month][day][f'Article_{"0" * (3-len(str(index+1)))}{index+1}'] = {
                    'Title': article_title,
                    'Description': article_description,
                    'Link': link['href'],
                }
    
        time.sleep(0.5)
        # break 

In [3]:
scrape_tempo_attraction_article()

'https://www.tempo.co/indeks/2024-01-09/travel'

Looping Each Dates:   0%|          | 0/3628 [00:00<?, ?it/s]

Looping Each Links of Dates: 2024-01-09:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
################################## EXPERIMENTAL STUFF for INEWS ##################################

In [18]:
def get_url_request(url):
    disable_warnings(InsecureRequestWarning)
    useragent_rotate = generate_user_agent()
    headers = {
        'User-Agent': useragent_rotate
    }
    
    response  = requests.get(url, headers=headers, verify=False)
    return response

In [43]:
url = 'https://www.inews.id/indeks/travel/{}-{}-{}'
df_article_information = pd.DataFrame(columns=['Year', 'Month', 'Day', 'Type'])

date_latest = datetime.date(2023, 2, 5)
date_early = datetime.date(2017, 10, 18)
dates = [date_latest - datetime.timedelta(days=x) for x in range((date_latest-date_early).days + 1)]

In [44]:
url_logger = display(display_id=True)
article_logger = display(display_id=True)
for date in tqdm(dates, desc='Looping Each Dates'):
    day, month, year = date.strftime('%d'), date.strftime('%m'), date.strftime('%Y')
    url_logger.update(url.format(year, month, day))

    response = get_url_request(url.format(day, month, year))
    soup = bs(response.text)
    soup_article_type = soup.select('#news-list > li > a > div > div > div.float-left.width-400px.margin-130px-left > div.date.margin-10px-left > strong')
    soup_article_links = soup.select('#news-list > li > a')

    if soup_article_links:
        for link, type in zip(soup_article_links, soup_article_type):
            article_information_dict = {
                'Year': year,
                'Month': month,
                'Day': day,
                'Link': link['href'] + '/all',
                'Type': type.text,
            }
            df_article_information = pd.concat([df_article_information, pd.DataFrame([article_information_dict.values()], columns=article_information_dict.keys())], ignore_index=True)

    time.sleep(0.3)
    # break

'https://www.inews.id/indeks/travel/2017-10-18'

Looping Each Dates:   0%|          | 0/1937 [00:00<?, ?it/s]

In [47]:
df_article_information.to_csv('./dataset/inews_article_attraction_part2.csv', index=False)

In [7]:
url_logger = display(display_id=True)
article_logger = display(display_id=True)
for date in tqdm(dates, desc='Looping Each Dates'):
    day, month, year = date.strftime('%d'), date.strftime('%m'), date.strftime('%Y')
    url_logger.update(url.format(year, month, day))

    response = get_url_request(url.format(day, month, year))
    soup = bs(response.text)
    soup_article_links = soup.select('#news-list > li > a')

    if soup_article_links:
        for index, link in enumerate(tqdm(soup_article_links, desc=f'Looping Each Links of Dates: {year}-{month}-{day}', leave=False)):
            link = link['href'] + '/all'
            article_logger.update(link)
            for i in range(15):
                article_response = get_url_request(link)
                if article_response.status_code == 200:
                    break
                elif article_response.status_code == 504:
                    time.sleep(0.3 * (i+1))
            article_soup = bs(article_response.text)

            if article_soup.select_one('#content > div > div.row > div.col-lg-9.col-md-8.col-sm-7.col-xs-9 > div > div.news-detail > div > div.col-lg-12.col-md-12.col-sm-12.col-xs-12 > h1'):
                article_title = article_soup.select_one('#content > div > div.row > div.col-lg-9.col-md-8.col-sm-7.col-xs-9 > div > div.news-detail > div > div.col-lg-12.col-md-12.col-sm-12.col-xs-12 > h1').text
            elif article_soup.select_one('#content > div > div.row > div.col-lg-9.col-md-8.col-sm-7.col-xs-9 > div > div.news-videos-detail > div > div.col-lg-12.col-md-12.col-sm-12.col-xs-12 > h1'):
                article_title = article_soup.select_one('#content > div > div.row > div.col-lg-9.col-md-8.col-sm-7.col-xs-9 > div > div.news-videos-detail > div > div.col-lg-12.col-md-12.col-sm-12.col-xs-12 > h1').text
            
            article_description = ""
            if article_soup.select('#content > div > div.row > div.col-lg-9.col-md-8.col-sm-7.col-xs-9 > div > div.news-detail > div > div.col-lg-10.col-md-10.col-sm-10.col-xs-10 > div.body-text-news > div.text-news.vidy-load > p'):
                description_element = article_soup.select('#content > div > div.row > div.col-lg-9.col-md-8.col-sm-7.col-xs-9 > div > div.news-detail > div > div.col-lg-10.col-md-10.col-sm-10.col-xs-10 > div.body-text-news > div.text-news.vidy-load > p')
            elif article_soup.select('#content > div > div.row > div.col-lg-9.col-md-8.col-sm-7.col-xs-9 > div > div.news-videos-detail > div > div.col-lg-10.col-md-10.col-sm-10.col-xs-10 > div.body-text-video.vidy-load > div.text-video > p'):
                description_element = article_soup.select('#content > div > div.row > div.col-lg-9.col-md-8.col-sm-7.col-xs-9 > div > div.news-videos-detail > div > div.col-lg-10.col-md-10.col-sm-10.col-xs-10 > div.body-text-video.vidy-load > div.text-video > p')
            for item in description_element:
                if not item.has_attr('class'):
                    article_description += item.text + ' '

            article_information_dict = {
                'Year': year,
                'Month': month,
                'Day': day,
                'Title': article_title,
                'Description': article_description,
                'Link': link,
            }
            df_article_information = pd.concat([df_article_information, pd.DataFrame([article_information_dict.values()], columns=article_information_dict.keys())], ignore_index=True)

    time.sleep(0.5)
    # break

'https://www.inews.id/indeks/travel/2013-02-05'

'https://www.inews.id/travel/belanja/3-tempat-wisata-belanja-di-jakarta-yang-kini-usianya-ratusan-tahun/all'

Looping Each Dates:   0%|          | 0/1717 [00:00<?, ?it/s]

Looping Each Links of Dates: 2017-10-18:   0%|          | 0/1 [00:00<?, ?it/s]

In [93]:
df_article_information

,Year,Month,Day,Title,Description,Link
0,2023,02,05,Menyantap Lontong Cap Go Meh Legendaris di Sol...,"SOLO, iNews.id - Lontong cap go meh tidak bole...",https://www.inews.id/multimedia/video/menyanta...
1,2023,02,05,Keseruan Berkemah di Subang Sambil Berswafoto ...,"SUBANG, iNews.id - Wisata mata air Cimincul in...",https://www.inews.id/multimedia/video/keseruan...
2,2023,02,05,"\n Majukan Pariwisata ASEAN, Sa...","JAKARTA, iNews.id - Menteri Pariwisata dan Eko...",https://www.inews.id/travel/destinasi/majukan-...
3,2023,02,05,"Mengunjungi Eks Tambang Emas Antam di Banten, ...","BANTEN, iNews.id - Eks tambang emas di Desa Ci...",https://www.inews.id/multimedia/video/mengunju...
4,2023,02,05,"Uniknya Produksi Kopi Khas Pantura Tuban, Biji...","TUBAN, iNews.id - Rumah di Desa Sawir, Tuban i...",https://www.inews.id/multimedia/video/uniknya-...
...,...,...,...,...,...,...
16457,2017,10,19,\n Deretan Pulau Terindah di Du...,JAKARTA – Status Filipina sebagai negara denga...,https://www.inews.id/travel/destinasi/deretan-...
16458,2017,10,19,\n Cuaca Dingin Paling Asyik Sa...,CUACA dingin selalu membuat perut terasa lapar...,https://www.inews.id/travel/kuliner/cuaca-ding...
16459,2017,10,19,\n Menelusuri Habitat Gajah yan...,"DUA tahun lalu, gajah bernama Yongki sempat me...",https://www.inews.id/travel/destinasi/menelusu...
16460,2017,10,19,\n Sajian Menyeramkan di Malam ...,"TIDAK lama lagi pesta Halloween akan dimulai, ...",https://www.inews.id/travel/kuliner/sajian-men...


In [ ]:
df_article_information.to_csv('./dataset/inews_article_attraction_part2', index=False)

In [ ]:
################################## EXPERIMENTAL STUFF for TEMPO ##################################

In [142]:
with open('./dataset/tempo_article_attraction_part1.json', 'r') as json_path:
    json_tempo1 = json.load(json_path)

In [143]:
df_tempo_article = pd.DataFrame(columns=['Year', 'Month', 'Day', 'Title', 'Description', 'Link'])

for year in tqdm(json_tempo1, desc='Looping Each Years', leave=False):
    # print(year)
    for month in tqdm(json_tempo1[year], desc='Looping Each Month', leave=False):
        # print(month)
        for day in tqdm(json_tempo1[year][month], desc='Looping Each Days', leave=False):
            # print(day)
            df_temp = pd.concat([pd.json_normalize(json_tempo1[year][month][day][article]) for article in json_tempo1[year][month][day]], keys=json_tempo1[year][month][day].keys())
            df_temp = df_temp.reset_index(drop=True)
            df_temp[['Year', 'Month', 'Day']] = [year, month, day]

            df_tempo_article = pd.concat([df_tempo_article, df_temp], axis=0, ignore_index=True)
            del df_temp
            
            # break
        # break
    # break

Looping Each Years:   0%|          | 0/4 [00:00<?, ?it/s]

Looping Each Month:   0%|          | 0/1 [00:00<?, ?it/s]

Looping Each Days:   0%|          | 0/9 [00:00<?, ?it/s]

Looping Each Month:   0%|          | 0/12 [00:00<?, ?it/s]

Looping Each Days:   0%|          | 0/31 [00:00<?, ?it/s]

...

Looping Each Days:   0%|          | 0/28 [00:00<?, ?it/s]

Looping Each Days:   0%|          | 0/31 [00:00<?, ?it/s]

Looping Each Month:   0%|          | 0/2 [00:00<?, ?it/s]

Looping Each Days:   0%|          | 0/31 [00:00<?, ?it/s]

Looping Each Days:   0%|          | 0/21 [00:00<?, ?it/s]

In [154]:
# df_tempo_article3 = df_tempo_article.copy()
df_tempo_article3

,Year,Month,Day,Title,Description,Link
0,2021,11,29,"Museum Taman Prasasti, Makam Istri Raffles hin...","TEMPO.CO, Jakarta - Museum Taman Prasasti meru...",https://travel.tempo.co/read/1533918/museum-ta...
1,2021,11,29,"5 Istilah Tingkat Kematangan Steak, Jangan Tah...","TEMPO.CO, Jakarta - Sebelum makan dan memesan...",https://travel.tempo.co/read/1533848/5-istilah...
2,2021,11,29,Rangkaian Destinasi Wisata Unggulan di Ternate...,"TEMPO.CO, Jakarta - Objek wisata di Ternate ta...",https://travel.tempo.co/read/1533833/rangkaian...
3,2021,11,29,Korea Selatan Menjadikan Pantai Busan Sebagai ...,"TEMPO.CO, Jakarta - Korea Selatan akan menjadi...",https://travel.tempo.co/read/1533819/korea-sel...
4,2021,11,29,"Libur Natal dan Tahun Baru, Petugas Kesehatan ...","TEMPO.CO, Jakarta - Pemerintah menerapkan Pemb...",https://travel.tempo.co/read/1533787/libur-nat...
...,...,...,...,...,...,...
12971,2018,01,19,Pesona Gunung Api dan Gua Bawah Laut Terpanjan...,"TEMPO.CO, Jakarta - Dunia ilmu pengetahuan bar...",https://travel.tempo.co/read/1052100/pesona-gu...
12972,2018,01,19,Kain Pantai dan Oleh-oleh Indonesia Mencuri Pe...,"TEMPO.CO, Jakarta - Indonesia kembali berparti...",https://travel.tempo.co/read/1052055/kain-pant...
12973,2018,01,19,Beragam Menu Melayu di Restoran Open Kitchen P...,"TEMPO.CO, Riau - Food Exchange All Day Dining ...",https://travel.tempo.co/read/1052064/beragam-m...
12974,2018,01,19,Malaysia Jadi Tujuan Wisata Pensiunan Terbaik ...,"TEMPO.CO, Jakarta - Malaysia terpilih sebagai ...",https://travel.tempo.co/read/1051975/malaysia-...


In [155]:
# df_tempo_article1 = df_tempo_article.copy()
df_tempo_article1

,Year,Month,Day,Title,Description,Link
0,2024,01,09,Dubai Destinasi Terpopuler di Dunia Menurut Tr...,"TEMPO.CO, Jakarta - Dubai terpilih menjadi des...",https://travel.tempo.co/read/1819232/dubai-des...
1,2024,01,09,"Tahun Ini Usia Cirebon Lebih Muda, Apa Sebabnya?","TEMPO.CO, Cirebon - Tahun ini Kota Cirebon ber...",https://travel.tempo.co/read/1819200/tahun-ini...
2,2024,01,09,"10 Wisata Religi di Surabaya, Ada Ziarah Wali ...","TEMPO.CO, Jakarta - Wisata religi bisa menjadi...",https://travel.tempo.co/read/1819156/10-wisata...
3,2024,01,09,"Mengenal Visa Digital Nomad, Keuntungan, dan C...","TEMPO.CO, Jakarta - Saat ini makin banyak oran...",https://travel.tempo.co/read/1819147/mengenal-...
4,2024,01,09,Brazil Menunda Persyaratan Visa untuk 3 Warga ...,"TEMPO.CO, Jakarta - Brazil menunda pemberlakua...",https://travel.tempo.co/read/1819083/brazil-me...
...,...,...,...,...,...,...
8819,2021,11,10,"Gulu Gulu Rilis Menu Cake & Cookie Collection,...","TEMPO.CO, Jakarta - Minuman cheese tea Gulu Gu...",https://travel.tempo.co/read/1527166/gulu-gulu...
8820,2021,11,10,Sistem Listrik World Superbike Sama Seperti Is...,"TEMPO.CO, Mataram - PT Perusahaan Listrik Nega...",https://travel.tempo.co/read/1527152/sistem-li...
8821,2021,11,10,12 Tarian Klasik Yogyakarta Tampil di Dubai Wo...,"TEMPO.CO, Yogyakarta - Pemerintah Daerah Istim...",https://travel.tempo.co/read/1527092/12-tarian...
8822,2021,11,10,"Melancong ke Shibuya Jepang, Tak Lengkap Jika ...","TEMPO.CO, Jakarta -Shibuya menjadi lokasi yang...",https://travel.tempo.co/read/1527054/melancong...


In [156]:
pd.concat([df_tempo_article1, df_tempo_article3], axis=0)

,Year,Month,Day,Title,Description,Link
0,2024,01,09,Dubai Destinasi Terpopuler di Dunia Menurut Tr...,"TEMPO.CO, Jakarta - Dubai terpilih menjadi des...",https://travel.tempo.co/read/1819232/dubai-des...
1,2024,01,09,"Tahun Ini Usia Cirebon Lebih Muda, Apa Sebabnya?","TEMPO.CO, Cirebon - Tahun ini Kota Cirebon ber...",https://travel.tempo.co/read/1819200/tahun-ini...
2,2024,01,09,"10 Wisata Religi di Surabaya, Ada Ziarah Wali ...","TEMPO.CO, Jakarta - Wisata religi bisa menjadi...",https://travel.tempo.co/read/1819156/10-wisata...
3,2024,01,09,"Mengenal Visa Digital Nomad, Keuntungan, dan C...","TEMPO.CO, Jakarta - Saat ini makin banyak oran...",https://travel.tempo.co/read/1819147/mengenal-...
4,2024,01,09,Brazil Menunda Persyaratan Visa untuk 3 Warga ...,"TEMPO.CO, Jakarta - Brazil menunda pemberlakua...",https://travel.tempo.co/read/1819083/brazil-me...
...,...,...,...,...,...,...
12971,2018,01,19,Pesona Gunung Api dan Gua Bawah Laut Terpanjan...,"TEMPO.CO, Jakarta - Dunia ilmu pengetahuan bar...",https://travel.tempo.co/read/1052100/pesona-gu...
12972,2018,01,19,Kain Pantai dan Oleh-oleh Indonesia Mencuri Pe...,"TEMPO.CO, Jakarta - Indonesia kembali berparti...",https://travel.tempo.co/read/1052055/kain-pant...
12973,2018,01,19,Beragam Menu Melayu di Restoran Open Kitchen P...,"TEMPO.CO, Riau - Food Exchange All Day Dining ...",https://travel.tempo.co/read/1052064/beragam-m...
12974,2018,01,19,Malaysia Jadi Tujuan Wisata Pensiunan Terbaik ...,"TEMPO.CO, Jakarta - Malaysia terpilih sebagai ...",https://travel.tempo.co/read/1051975/malaysia-...


In [161]:
df_tempo_full = pd.concat([df_tempo_article1, df_tempo_article3], axis=0).drop_duplicates('Link', keep='last').sort_values(['Year', 'Month', 'Day'], ascending=False).reset_index(drop=True)

In [162]:
df_tempo_full.head()

,Year,Month,Day,Title,Description,Link
0,2024,01,09,Dubai Destinasi Terpopuler di Dunia Menurut Tr...,"TEMPO.CO, Jakarta - Dubai terpilih menjadi des...",https://travel.tempo.co/read/1819232/dubai-des...
1,2024,01,09,"Tahun Ini Usia Cirebon Lebih Muda, Apa Sebabnya?","TEMPO.CO, Cirebon - Tahun ini Kota Cirebon ber...",https://travel.tempo.co/read/1819200/tahun-ini...
2,2024,01,09,"10 Wisata Religi di Surabaya, Ada Ziarah Wali ...","TEMPO.CO, Jakarta - Wisata religi bisa menjadi...",https://travel.tempo.co/read/1819156/10-wisata...
3,2024,01,09,"Mengenal Visa Digital Nomad, Keuntungan, dan C...","TEMPO.CO, Jakarta - Saat ini makin banyak oran...",https://travel.tempo.co/read/1819147/mengenal-...
4,2024,01,09,Brazil Menunda Persyaratan Visa untuk 3 Warga ...,"TEMPO.CO, Jakarta - Brazil menunda pemberlakua...",https://travel.tempo.co/read/1819083/brazil-me...


In [163]:
df_tempo_full.to_csv('./dataset/tempo_article_attraction.csv', index=False)

In [2]:
def get_url_request(url):
    useragent_rotate = generate_user_agent()
    headers = {
        'User-Agent': useragent_rotate
    }
    
    response  = requests.get(url, headers=headers)
    return response

In [3]:
url = 'https://www.tempo.co/indeks/{}-{}-{}/travel'
article_information_dict = {}

date_latest = datetime.date(2021, 11, 29)
date_early = datetime.date(2014, 1, 9)
dates = [date_latest - datetime.timedelta(days=x) for x in range((date_latest-date_early).days + 1)]

In [4]:
url_logger = display(display_id=True)
for date in tqdm(dates, desc='Looping Each Dates'):
    day, month, year = date.strftime('%d'), date.strftime('%m'), date.strftime('%Y')
    article_logger = display(display_id=True)
    url_logger.update(url.format(year, month, day))

    response = get_url_request(url.format(year, month, day))
    soup = bs(response.text)
    soup_article_links = soup.select('body > main > div.container.small.left.padding-no > main > div.card-box.ft240.margin-bottom-sm > article > h2 > a')

    if soup_article_links:
        article_information_dict.setdefault(year, {})\
            .setdefault(month, {})\
            .setdefault(day, {})
        for index, link in enumerate(tqdm(soup_article_links, desc=f'Looping Each Links of Dates: {year}-{month}-{day}', leave=False)):
            for i in range(15):
                article_response = get_url_request(link['href'])
                if article_response.status_code == 200:
                    break
                elif article_response.status_code == 504:
                    time.sleep(0.5 * (i+1))
            article_soup = bs(article_response.text)

            article_title = article_soup.select_one('body > main > div.container.small.left.padding-no > main > article.detail-artikel > div.detail-title > h1').text
            article_description = ""
            for item in article_soup.select('#isi > div > div.detail-konten > p'):
                article_description += item.text + ' '

            article_information_dict[year][month][day][f'Article_{"0" * (3-len(str(index+1)))}{index+1}'] = {
                'Title': article_title,
                'Description': article_description,
                'Link': link['href'],
            }

    time.sleep(0.5)
    # break 

'https://www.tempo.co/indeks/2014-01-09/travel'

Looping Each Dates:   0%|          | 0/2882 [00:00<?, ?it/s]

Looping Each Links of Dates: 2021-11-29:   0%|          | 0/9 [00:00<?, ?it/s]

Looping Each Links of Dates: 2021-11-28:   0%|          | 0/13 [00:00<?, ?it/s]

Looping Each Links of Dates: 2021-11-27:   0%|          | 0/9 [00:00<?, ?it/s]

...

Looping Each Links of Dates: 2018-01-22:   0%|          | 0/7 [00:00<?, ?it/s]

Looping Each Links of Dates: 2018-01-21:   0%|          | 0/9 [00:00<?, ?it/s]

Looping Each Links of Dates: 2018-01-20:   0%|          | 0/7 [00:00<?, ?it/s]

Looping Each Links of Dates: 2018-01-19:   0%|          | 0/4 [00:00<?, ?it/s]

Looping Each Links of Dates: 2017-11-17:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
print(json.dumps(article_information_dict, sort_keys=True, indent=4))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [29]:
type(article_response.status_code)

int

In [17]:
article_response.status_code

int

In [6]:
with open('./dataset/tempo_article_attraction_part3.json', 'w') as json_file:
    json.dump(article_information_dict, json_file, indent=4)

In [7]:
list(article_information_dict)[-1]

'2017'

In [8]:
list(article_information_dict['2021'])[-1]

'01'

In [9]:
list(article_information_dict['2021']['11'])[-1]

'01'

In [ ]:
################################## EXPERIMENTAL STUFF ##################################

In [10]:
url = 'https://www.tempo.co/indeks/{}-{}-{}/travel'

In [11]:
response = requests.post(url)

In [14]:
soup = bs(response.text)

In [15]:
soup

<!DOCTYPE html>
<html class="no-js" lang="en">
<head>
<title>Indeks 1 Februari 2024</title>
<meta content="332404380172618" property="fb:app_id"/>
<meta content="id_ID" property="og:locale"/>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://www.tempo.co/indeks/2024-02-01/travel" rel="original-source"/>
<link href="https://plus.google.com/109335234362909335582" rel="publisher"/>
<meta content="" name="description"/>
<meta content="" name="keywords"/>
<meta content="all" name="robots"/>
<meta content="index, follow, max-image-preview:large" name="robots"/>
<meta content="index, follow" name="yahoobot"/>
<meta content="index, follow" name="googlebot-news"/>
<meta content="index, follow" name="googlebot"/>
<link href="https://rss.tempo.co/" rel="alternate" title="Tempo.co RSS Feed" type="application/rss+xml"/>
<meta content="M2xcdJ5TZd5mAsle_IY5qEv42TAd08Jkheo4-7tovx4" 

In [25]:
soup_article_links = soup.select('body > main > div.container.small.left.padding-no > main > div.card-box.ft240.margin-bottom-sm > article > h2 > a')

In [27]:
for item in soup_article_links:
    print(item['href'])

https://travel.tempo.co/read/1828650/traveloka-bagi-rekomendasi-manfaatkan-libur-tahun-baru-imlek-dan-pemilu
https://travel.tempo.co/read/1828580/itdc-dan-pemkab-lombok-tengah-optimalisasi-aset-di-kek-mandalika
https://travel.tempo.co/read/1828641/manfaatkan-promo-liburan-untuk-tahun-baru-imlek-dan-setelah-pemilu
https://travel.tempo.co/read/1828630/working-holiday-visa-australia-peluang-liburan-panjang-dan-bekerja-di-negeri-kanguru-ketahui-syaratnya
https://travel.tempo.co/read/1828581/asal-usul-karnaval-venesia-dan-deretan-acaranya
https://travel.tempo.co/read/1828545/kapal-pesiar-italia-aida-bella-singgah-di-pelabuhan-gili-mas-lombok
https://travel.tempo.co/read/1828517/strategi-agar-bali-bisa-jaring-7-juta-wisatawan-mancanegara-tanpa-overtourism
https://travel.tempo.co/read/1828474/marak-aksi-boikot-starbucks-indonesia-sebut-toko-lebih-sepi-dan-mitra-terdampak
https://travel.tempo.co/read/1828441/akses-machu-picchu-dibuka-lagi-setelah-aksi-protes-berakhir
https://travel.tempo.co/re

In [47]:
article_information_dict = {}

In [56]:
d1 = datetime.date(2015, 1, 1)
d2 = datetime.date(2016, 2, 6)
days = [d1 + datetime.timedelta(days=x) for x in range((d2-d1).days + 1)]

for day in days:
    article_information_dict.setdefault(day.strftime('%Y'), {})\
        .setdefault(day.strftime('%m'), {})\
        .setdefault(day.strftime('%d'), {})
    print(f'day: {day.strftime("%Y")} | month: {day.strftime("%m")} | year: {day.strftime("%d")}')

    
    
    # break

day: 2015 | month: 01 | year: 01
day: 2015 | month: 01 | year: 02
day: 2015 | month: 01 | year: 03
day: 2015 | month: 01 | year: 04
day: 2015 | month: 01 | year: 05
day: 2015 | month: 01 | year: 06
day: 2015 | month: 01 | year: 07
day: 2015 | month: 01 | year: 08
day: 2015 | month: 01 | year: 09
day: 2015 | month: 01 | year: 10
day: 2015 | month: 01 | year: 11
day: 2015 | month: 01 | year: 12
day: 2015 | month: 01 | year: 13
day: 2015 | month: 01 | year: 14
day: 2015 | month: 01 | year: 15
day: 2015 | month: 01 | year: 16
day: 2015 | month: 01 | year: 17
day: 2015 | month: 01 | year: 18
day: 2015 | month: 01 | year: 19
day: 2015 | month: 01 | year: 20
day: 2015 | month: 01 | year: 21
day: 2015 | month: 01 | year: 22
day: 2015 | month: 01 | year: 23
day: 2015 | month: 01 | year: 24
day: 2015 | month: 01 | year: 25
day: 2015 | month: 01 | year: 26
day: 2015 | month: 01 | year: 27
day: 2015 | month: 01 | year: 28
day: 2015 | month: 01 | year: 29
day: 2015 | month: 01 | year: 30
day: 2015 

In [220]:
url = "https://travel.tempo.co/read/1819232/dubai-destinasi-terpopuler-di-dunia-menurut-tripadvisor-bali-kedua"

In [221]:
response = get_url_request(url)

In [222]:
soup = bs(response.text)

In [223]:
article_title = soup.select_one('body > main > div.container.small.left.padding-no > main > article.detail-artikel > div.detail-title > h1').text
article_title

'Dubai Destinasi Terpopuler di Dunia Menurut Tripadvisor, Bali Kedua'

In [201]:
article_description = ""
for item in soup.select('#isi > div > div.detail-konten > p'):
    article_description += item.text + ' '

In [203]:
article_description

'TEMPO.CO, Jakarta - Dubai terpilih menjadi destinasi terpopuler di dunia menurut Traveller\'s Choice Tripadvisor Awards 2024. Ini adalah tahun ketiga Dubai mendapatkan penghargaan tersebut.\xa0 Traveller\'s Choice TripAdvisor Awards tahun 2024 dipilih berdasarkan kualitas dan kuantitas ulasan serta penilaian khusus wisatawan. Ada beberapa subkategori yang dinilai termasuk akomodasi, restoran, dan aktivitas. Penilaian ini dilakukan seluruh destinasi di seluruh dunia selama 12 bulan antara Oktober 2022 hingga September 2023. Menurut TripAdvisor, Dubai memiliki daya tarik yang spektakuler bagi wisatawan. Mulai dari Burj Khalifa yang ultra-modern hingga pasar-pasar dan mal yang dipenuhi penjual emas dan perhiasan.\xa0 "Ini adalah tempat di mana jika Anda bisa memimpikannya, Anda bisa melakukannya, apakah itu berarti bermain ski di dalam ruangan, berselancar di bukit pasir di padang pasir, atau bermain zip-lining di atas kota," kata Tripadvisor.\xa0 Sementara peringkat kedua adalah Bali, I